# Theory

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import facets_drivers

%store -r notebook_data
globals().update(notebook_data)

In [ ]:
if not local_run:
    from ewoksdask.schedulers import slurm_scheduler
    cluster = slurm_scheduler(account="staff-id01",queue="nice", walltime="01:00:00", memory="40GB", cores=num_cores)

In [ ]:
from ewoksdask import execute_graph

nodes = [
    {
        "id": "get_facets",
        "task_type": "class",
        "task_identifier": "facets_drivers.theory.GetFacets",
    },
    {
        "id": "get_orientation",
        "task_type": "class",
        "task_identifier": "facets_drivers.theory.GetOrientation",
    },
    {
        "id": "get_oriented_facets",
        "task_type": "class",
        "task_identifier": "facets_drivers.theory.GetOrientedFacets",
    },
]

links = [
    {
        "source": "get_facets",
        "target": "get_orientation",
        "data_mapping": [{"source_output": "vtp_facets_filepath", "target_input": "vtp_facets_filepath"},
                         {"source_output": "scratch_directory", "target_input": "scratch_directory"}],
    },
    {
        "source": "get_orientation",
        "target": "get_oriented_facets",
        "data_mapping": [{"source_output": "vtp_facets_filepath", "target_input": "vtp_facets_filepath"},
                         {"source_output": "orientation_filepath", "target_input": "orientation_filepath"},
                         {"source_output": "scratch_directory", "target_input": "scratch_directory"}],
    },
]

workflow = {"graph": {"id": "theroy"}, "nodes": nodes, "links": links}

positive_hkl = ",".join(str(i) for i in hkl)
negative_hkl = ",".join(str(-i) for i in hkl)
exclude_hkl = f"{positive_hkl};{negative_hkl}"
inputs = [
    {"id": "get_facets", "name": "scratch_directory", "value": theory_directory},
    {"id": "get_facets", "name": "vti_filepath", "value": experimental_structure},
    {"id": "get_facets", "name": "scale_factors", "value": scale_factors},
    {"id": "get_orientation", "name": "hkl", "value": "".join(str(i) for i in hkl)},
    {"id": "get_orientation", "name": "exclude_hkl", "value": exclude_hkl},
    {"id": "get_oriented_facets", "name": "scale_factors", "value": scale_factors},
]

In [ ]:
%%time
result = execute_graph(workflow, inputs=inputs, scheduler=None if local_run else cluster.scheduler_address)

In [ ]:
print(result)

In [ ]:
if not local_run:
    cluster.close()